In [46]:
import pandas as pd
import numpy as np
import pymorphy2
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier,RidgeClassifierCV,LogisticRegressionCV, SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize,TreebankWordTokenizer
import re
import nltk
from nltk.corpus import wordnet
from emoji.unicode_codes import UNICODE_EMOJI
import emoji
from nltk.corpus import stopwords

In [6]:
comments = pd.read_csv('28_task_result.csv', sep = ';', names = ['comment', 'level', 'label'])

In [8]:
comments.head()

,comment,level,label
0,Спасибо !,simpleTree,общеразговорные
1,Ставропольский край . г Ессентуки . ул Октябрь...,simpleTree,вопросы о сервисе
2,Здравствуйте **** . Можно ли заказать следующи...,simpleTree,вопросы о товаре
3,Спасибо ),simpleTree,общеразговорные
4,Можете сказать фирму блузы ?,simpleTree,вопросы о товаре


In [10]:
comments.label.value_counts()/comments.shape[0]

другие               0.49600
вопросы о сервисе    0.19150
вопросы о товаре     0.14450
общеразговорные      0.14175
более 1 категории    0.02625
Name: label, dtype: float64

In [127]:
comments['label_num'] = comments.label.map({'другие':0, 'вопросы о сервисе':1, 'вопросы о товаре':2, 'общеразговорные':3, 'более 1 категории':4})

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer(ngram_range = (1,1), analyzer = 'word',
                       stop_words = 'english',
                       max_features = 500,
                       min_df = 2, max_df = 0.95).fit(comments.comment)
pw = list(vect.vocabulary_.keys())

In [12]:
with open("pw.txt", "w") as text_file:
    for w in pw:
        print(w, file=text_file)

In [110]:
def check_spelling(text):
    if not c.check(text):
        suggestions = list(set(c.suggest(text)))#.intersection(set(pw)))
        if len(suggestions)>0:
            res = suggestions[0]
        elif len(c.suggest(text))>0:
            res = c.suggest(text)[0]
        else:
            res = text
    else:
        res = text
    return res    

In [112]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def clean_comment(text):
    wnl = WordNetLemmatizer()
    tokenizer = TreebankWordTokenizer()
    text = re.sub(r'\W',' ', text)
    tokens = tokenizer.tokenize(text)
    tags = nltk.pos_tag(tokens)
    processed = []
    #processed = [wnl.lemmatize(check_spelling(word),get_wordnet_pos(tag)) if get_wordnet_pos(tag)!='' else wnl.lemmatize(check_spelling(word)) for (word,tag) in tags]
    for (word, tag) in tags:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag!='':
            processed.append(wnl.lemmatize(check_spelling(word),wn_tag))
        else:
            processed.append(wnl.lemmatize(check_spelling(word)))
    res = ' '.join(processed)
    return res.lower()

In [113]:
import time

In [114]:
start_time = time.time()
clean_comment('Заплоти пес')
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.11315321922302246 seconds ---


In [115]:
comments['cleaned'] = comments.comment.apply(clean_comment)

In [116]:
comments.cleaned.head()

0                                              спасибо
1    ставропольский край гр ессентуки ул октябрьска...
2    здравствуйте можно ли заказать следующие модел...
3                                              спасибо
4                           можете сказать фирму блузы
Name: cleaned, dtype: object

In [135]:
comments.cleaned.iloc[100]

'здравствуйте у вас есть номер отслеживания моего заказа что то не пришел еще думаю может на почте спросить по номеру'

In [41]:
#comments.to_csv('comments_cleaned.csv')

In [133]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LassoCV
vect = CountVectorizer()
model = LogisticRegressionCV()
scaler = MaxAbsScaler()
stop_words = stopwords.words('russian')
lin_model = Pipeline([#('vectorizer', vect),
                      ('features', FeatureUnion([
                        ('words', CountVectorizer(ngram_range = (1,3), max_features = 500,analyzer = 'word', stop_words = stop_words)),
                        ('chars', CountVectorizer(ngram_range = (1,5), max_features = 500,analyzer = 'char', stop_words = stop_words)),
                      ])),
                        #('scaler', scaler),
                      ('classifier', model)])
lin_model.set_params(classifier__class_weight = 'balanced')#,classifier__alphas = np.linspace(start = 0.01, stop = 100, num = 50))
feats = comments.cleaned
labels = comments.label_num
lin_model = lin_model.fit(feats, labels)
print('cross_val_score: %1.4f'% (np.mean(cross_val_score(lin_model, feats,labels))))

cross_val_score: 0.6333


In [138]:
def eval_pipeline(input_text, model = lin_model):
    cleaned_text = clean_comment(input_text)
    class_names = ['другие',
        'вопросы о сервисе',
        'вопросы о товаре',
        'общеразговорные',
        'более 1 категории']
    prediction = model.predict([cleaned_text])
    #print(class_names[prediction[0]])
    return class_names[prediction[0]]

In [146]:
eval_pipeline('Здравствуйте. У вас есть синие комбинезоны? Адрес: г. Новочеркасск, ул. Щорса 83, кв. 57')

'вопросы о товаре'